# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 28 2022 7:11PM,249643,21,631484,"NBTY Global, Inc.",Released
1,Oct 28 2022 7:10PM,249642,21,631913,"NBTY Global, Inc.",Released
2,Oct 28 2022 7:10PM,249641,21,631412,"NBTY Global, Inc.",Released
3,Oct 28 2022 6:18PM,249639,21,IR102522,"NBTY Global, Inc.",Released
4,Oct 28 2022 4:57PM,249638,21,RE102822,"NBTY Global, Inc.",Released
5,Oct 28 2022 3:40PM,249634,20,ATR-I000015,"HVL, LLC dba Atrium Innovations",Released
6,Oct 28 2022 10:16AM,249594,12,8703768,"SD Head USA, LLC",Released
7,Oct 28 2022 9:18AM,249587,16,8700664,Sartorius Lab Products and Service,Completed
8,Oct 28 2022 9:18AM,249587,16,8700668,Sartorius Lab Products and Service,Released
9,Oct 28 2022 9:18AM,249587,16,8700671,Sartorius Lab Products and Service,Completed


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
4,249638,Released,1
5,249639,Released,1
6,249641,Released,1
7,249642,Released,1
8,249643,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Released
id,,
249638,NaN,1.0
249639,NaN,1.0
249641,NaN,1.0
249642,NaN,1.0
249643,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Released
id,,
249587,4.0,1.0
249594,0.0,1.0
249634,0.0,1.0
249638,0.0,1.0
249639,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,Completed,Released
id,,
249587,4,1
249594,0,1
249634,0,1
249638,0,1
249639,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Released
0,249587,4,1
1,249594,0,1
2,249634,0,1
3,249638,0,1
4,249639,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Executing does not exist yet


ShipmentStatus,id,Completed,Released
0,249587,4,1
1,249594,,1
2,249634,,1
3,249638,,1
4,249639,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc."
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc."
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc."
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc."
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc."
5,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations"
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC"
7,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Released
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc.",,1
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc.",,1
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc.",,1
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc.",,1
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",,1
5,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations",,1
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",,1
7,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,4,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc.",1,
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc.",1,
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc.",1,
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc.",1,
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,
5,Oct 28 2022 3:40PM,249634,20,"HVL, LLC dba Atrium Innovations",1,
6,Oct 28 2022 10:16AM,249594,12,"SD Head USA, LLC",1,
7,Oct 28 2022 9:18AM,249587,16,Sartorius Lab Products and Service,1,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc.",1,
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc.",1,
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc.",1,
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc.",1,
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet


,Date,id,Warehouse,Customer,Released,Completed
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc.",1,NaN
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc.",1,NaN
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc.",1,NaN
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc.",1,NaN
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Completed
0,Oct 28 2022 7:11PM,249643,21,"NBTY Global, Inc.",1,0.0
1,Oct 28 2022 7:10PM,249642,21,"NBTY Global, Inc.",1,0.0
2,Oct 28 2022 7:10PM,249641,21,"NBTY Global, Inc.",1,0.0
3,Oct 28 2022 6:18PM,249639,21,"NBTY Global, Inc.",1,0.0
4,Oct 28 2022 4:57PM,249638,21,"NBTY Global, Inc.",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Completed
Warehouse,,,
12,249594,1,0.0
16,249587,1,4.0
20,249634,1,0.0
21,1248203,5,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Completed
0,12,249594,1,0.0
1,16,249587,1,4.0
2,20,249634,1,0.0
3,21,1248203,5,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Completed
0,12,1,0.0
1,16,1,4.0
2,20,1,0.0
3,21,5,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,12,Released,1.0
1,16,Released,1.0
2,20,Released,1.0
3,21,Released,5.0
4,12,Completed,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,12,16,20,21
Status,,,,
Completed,0.0,4.0,0.0,0.0
Released,1.0,1.0,1.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,12,16,20,21
0,Completed,0.0,4.0,0.0,0.0
1,Released,1.0,1.0,1.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,12,16,20,21
0,Completed,0.0,4.0,0.0,0.0
1,Released,1.0,1.0,1.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()